In [1]:
# prerequisite
# pip install -U imbalanced-learn

In [2]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from utils.Common import Config
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE

import pandas as pd
import numpy as np
import joblib

In [3]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [4]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [5]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [6]:
# pass feature to pipeline and convert it to numerical data
dc = DataPipeline(Config.num_attribs,Config.cat_attribs)
X = dc.process(X)

In [7]:
X.isna().sum().sum()

0

In [8]:
Y.value_counts().tolist()

[14246, 2201]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y)

In [10]:
smote_minority = SMOTE(n_jobs = -1, sampling_strategy = "minority")
X_train_sm, y_train_sm = smote_minority.fit_resample(X_train, y_train)


c:\Users\yvonn\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [11]:


rf_clf = joblib.load('../models/best_model_random_forest.pkl')
svc_clf = joblib.load('../models/best_model_svc.pkl')
knn_clf = joblib.load('../models/best_model_knn.pkl')


In [12]:
rf_clf.fit(X_train_sm, y_train_sm)

RandomForestClassifier(max_depth=14, min_samples_leaf=4, min_samples_split=25)

In [13]:
rf_clf.score(X_train_sm, y_train_sm)

0.9380484380484381

In [14]:
rf_clf.score(X_test,y_test)

0.8680851063829788

In [15]:
svc_clf.fit(X_train_sm, y_train_sm)


SVC(C=100, class_weight='balanced')

In [16]:
svc_clf.score(X_train_sm, y_train_sm)

0.9998683748683749

In [17]:
svc_clf.score(X_test,y_test)

0.9133738601823708

In [18]:
knn_clf.fit(X_train_sm, y_train_sm)

KNeighborsClassifier(metric='manhattan', weights='distance')

In [19]:
knn_clf.score(X_train_sm, y_train_sm)

1.0

In [20]:
knn_clf.score(X_test,y_test)

0.9066869300911854

In [21]:

vt_hard_clf = VotingClassifier(estimators= [
         ('rf', rf_clf), 
         ('svc', svc_clf), 
         ('knn', knn_clf)
         ]
, voting='hard')

In [22]:
vt_hard_clf.fit(X_train_sm, y_train_sm)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=14,
                                                     min_samples_leaf=4,
                                                     min_samples_split=25)),
                             ('svc', SVC(C=100, class_weight='balanced')),
                             ('knn',
                              KNeighborsClassifier(metric='manhattan',
                                                   weights='distance'))])

In [23]:
vt_hard_clf.score(X_train_sm, y_train_sm)

0.9999122499122499

In [24]:
vt_hard_clf.score(X_test,y_test)

0.9240121580547113

In [28]:
vt_soft_clf = VotingClassifier(estimators= [
         ('rf', rf_clf), 
         ('svc', svc_clf), 
         ('knn', knn_clf)
         ]
, voting='soft')
vt_soft_clf.named_estimators['svc'].probability=True

In [29]:
vt_soft_clf.fit(X_train_sm, y_train_sm)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=14,
                                                     min_samples_leaf=4,
                                                     min_samples_split=25)),
                             ('svc',
                              SVC(C=100, class_weight='balanced',
                                  probability=True)),
                             ('knn',
                              KNeighborsClassifier(metric='manhattan',
                                                   weights='distance'))],
                 voting='soft')

In [30]:
vt_soft_clf.score(X_train_sm, y_train_sm)

1.0

In [31]:
vt_soft_clf.score(X_test,y_test)

0.9300911854103343